In [2]:
import pandas as pd
from pandas import DataFrame
import re
import gensim
import gensim.corpora as corpora
from typing import Generator, List
import spacy
from pprint import pprint

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from spacy.lang.en import English

import nltk
from nltk.corpus import stopwords  

nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [6]:
def read_sample() -> DataFrame:
    df = pd.read_json('C:/Users/chiruco/Desktop/python/ProyPython/Topic_Model/Topic_model_py/data/raw/newsgroups.json')
    return df

nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

def clean_up_text(df:pd.DataFrame) -> List[str]:
    data = df.content.values.tolist() # convertir a lista
    data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data] # Quitar e-mail
    data = [re.sub(r'\s+', ' ', sent) for sent in data] # quitar enters (new line)
    data = [re.sub(r"\'", "", sent) for sent in data] # quitar comillas
    return(data)

def sentence_to_words(sentences: List[str]) -> List[List[str]]:
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))    

def remove_stopwords(texts: List[List[str]]) -> List[List[str]]:
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def lemmatization(npl: English, texts:List[List[str]], allowed_postags:List[str]=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


def core_bigram(data_words: List[List[str]], min_count:int=5,threshold:int=10):
    bigram = gensim.models.Phrases(data_words, min_count=min_count, threshold=threshold)
    return(bigram)

def build_bigrams(data_words: List[List[str]], min_count:int=5,threshold:int=10) -> List[List[str]]:
    bigram = core_bigram(data_words,min_count,threshold)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in data_words]
    
def build_trigrams(data_words:List[List[str]],min_count:int=5,threshold:int=10) -> List[List[str]]:
    bigram = core_bigram(data_words, min_count, threshold)
    trigram = gensim.models.Phrases(bigram[data_words], threshold)  
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return[trigram_mod[bigram_mod[doc]] for doc in data_words]

def load_doc() -> pd.DataFrame:
    return read_sample()

def lda_model(raw_file: pd.DataFrame):
    doc       = clean_up_text(raw_file)
    lemma     = tokenize(doc)
    id2word, corpus = create_dictionary(lemma)
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=20, random_state=100,
                                    update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)
    return(lda_model)

def tokenize(documents: List[str]) -> List[List[str]]:
    document_words = list(sentence_to_words(documents))
    document_words = remove_stopwords(document_words)
    document_words = build_bigrams(document_words)
    document_words = lemmatization(nlp, document_words)
    return document_words

def create_dictionary(documents: List[List[str]]):
    id2word = corpora.Dictionary(documents)
    texts   = documents
    corpus  = [id2word.doc2bow(text) for text in texts ]
    return (id2word,  corpus)

In [7]:
df              = load_doc()
doc             = clean_up_text(df)
lemma           = tokenize(doc)
id2word, corpus = create_dictionary(lemma)
lda_model       = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=20, random_state=100,
                  update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

In [250]:
pprint(lda_model.print_topics())

[(0,
  '0.118*"space" + 0.046*"launch" + 0.043*"mount" + 0.036*"orbit" + '
  '0.034*"satellite" + 0.033*"mission" + 0.028*"earth" + 0.027*"cool" + '
  '0.025*"moon" + 0.022*"fuel"'),
 (1,
  '0.064*"patient" + 0.032*"kill" + 0.031*"headache" + 0.030*"disease" + '
  '0.028*"soldier" + 0.027*"civilian" + 0.026*"murder" + 0.023*"village" + '
  '0.019*"turkish" + 0.018*"treatment"'),
 (2,
  '0.119*"sell" + 0.119*"price" + 0.051*"soon" + 0.031*"market" + 0.030*"pack" '
  '+ 0.030*"insurance" + 0.028*"communication" + 0.022*"cap" + 0.022*"ensure" '
  '+ 0.021*"agent"'),
 (3,
  '0.026*"people" + 0.014*"say" + 0.014*"may" + 0.013*"reason" + '
  '0.012*"believe" + 0.011*"evidence" + 0.010*"mean" + 0.009*"fact" + '
  '0.009*"state" + 0.009*"claim"'),
 (4,
  '0.076*"gun" + 0.041*"week" + 0.036*"fire" + 0.033*"kill" + 0.031*"carry" + '
  '0.026*"city" + 0.025*"police" + 0.025*"weapon" + 0.024*"office" + '
  '0.023*"shoot"'),
 (5,
  '0.037*"program" + 0.030*"file" + 0.024*"window" + 0.021*"available

In [227]:
data_words = list(sentence_to_words(data))
data_words = build_bigrams(data_words, 5,10)
#data_words = build_trigrams(data_words, 5,10)
data_words_nostops = remove_stopwords(data_words)

In [199]:
data_lemmatized = lemmatization(nlp, data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [201]:
print(data_lemmatized[:1])

[['where', 'car', 'nntp', 'post', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood']]


In [212]:
def create_dictionary(documents: List[List[str]]):
    id2word = corpora.Dictionary(documents)
    texts   = documents
    corpus  = [id2word.doc2bow(text) for text in texts ]
    return (id2word, texts, corpus)

In [214]:
id2word, texts, corpus = create_dictionary(data_lemmatized)

In [215]:
for key, value in id2word.items():
    print(key, value)

0 addition
1 body
2 bring
3 bumper
4 call
5 car
6 could
7 day
8 door
9 early
10 engine
11 enlighten
12 front
13 funky
14 history
15 info
16 know
17 late
18 line
19 look
20 mail
21 make
22 model
23 name
24 neighborhood
25 nntp
26 post
27 production
28 really
29 rest
30 see
31 separate
32 small
33 spec
34 sport
35 thank
36 where
37 wonder
38 year
39 acceleration
40 adapter
41 add
42 answer
43 article
44 attain
45 base
46 brave
47 brief
48 card
49 clock
50 detail
51 disk
52 do
53 especially
54 experience
55 final
56 floppy
57 functionality
58 guy
59 heat
60 hour
61 keyword
62 knowledge
63 message
64 network
65 next
66 number
67 oscillator
68 poll
69 procedure
70 rate
71 report
72 request
73 send
74 share
75 si
76 sink
77 soul
78 speed
79 summarize
80 summary
81 top
82 upgrade
83 usage
84 access
85 active
86 actually
87 advance
88 anymore
89 appearence
90 back
91 be
92 bite
93 breifly
94 bunch
95 computer
96 conviction
97 daily
98 dangerous
99 dirt
100 display
101 distribution
102 drop
103

1308 thus
1309 unhappy
1310 unit
1311 useless
1312 warranty
1313 acne
1314 advice
1315 anti
1316 appearance
1317 bury
1318 cap
1319 chemist
1320 chin
1321 counter
1322 cradle
1323 dandruff
1324 diet
1325 disfigure
1326 doctor
1327 escalate
1328 father
1329 flogging
1330 greasy
1331 hairline
1332 improve
1333 incline
1334 kid
1335 malady
1336 morale
1337 neighbour
1338 odd
1339 ointment
1340 overdose
1341 pharmacist
1342 prescription
1343 scaliness
1344 scalp
1345 senstitive
1346 serious
1347 severe
1348 shall
1349 shampoo
1350 son
1351 spot
1352 spotty
1353 teenage
1354 teenager
1355 vitamin
1356 wary
1357 wash
1358 wierd
1359 domestic
1360 explode
1361 television
1362 union
1363 act
1364 arab
1365 arguable
1366 army
1367 autumn
1368 battle
1369 begin
1370 bomb
1371 citizenship
1372 constant
1373 context
1374 cut
1375 desert
1376 easy
1377 elsewhere
1378 entirely
1379 expel
1380 fight
1381 fighting
1382 force
1383 grant
1384 grief
1385 imho
1386 intention
1387 invade
1388 invasion
1389

2528 revolutionary
2529 reward
2530 ringleader
2531 row
2532 sabotage
2533 saintly
2534 savage
2535 savagely
2536 scene
2537 scholar
2538 screw
2539 shocking
2540 slaughterhouse
2541 soldier
2542 spanish
2543 specially
2544 spell
2545 spurious
2546 squad
2547 stab
2548 stage
2549 stone
2550 street
2551 strike
2552 subject
2553 submit
2554 suffer
2555 summarily
2556 super
2557 superior
2558 supervise
2559 survive
2560 survivor
2561 systematic
2562 tartar
2563 tear
2564 terminate
2565 territorial
2566 terror
2567 terrorism
2568 text
2569 thunder
2570 tragedy
2571 traitorously
2572 treachery
2573 treasonable
2574 tree
2575 triangle
2576 troop
2577 uncultivated
2578 unfortunately
2579 urge
2580 urgently
2581 victory
2582 vilayet
2583 volunteer
2584 wire
2585 withdrawal
2586 witness
2587 attribution
2588 defend
2589 discussion
2590 forum
2591 glad
2592 harmless
2593 honest
2594 limit
2595 potentially
2596 prank
2597 punk
2598 taunt
2599 ugly
2600 vandalism
2601 alternative
2602 amp
2603 cam

3960 sketchiness
3961 testament
3962 roster
3963 brochure
3964 microwave
3965 blade
3966 blazer
3967 bull
3968 hardly
3969 instruction
3970 oakley
3971 orange
3972 perfect
3973 terminator
3974 advise
3975 ahead
3976 award
3977 mislable
3978 refund
3979 telephone
3980 visual
3981 digitally
3982 preset
3983 tune
3984 bi
3985 clue
3986 dramatically
3987 flawed
3988 hatemonger
3989 impose
3990 incredibly
3991 median
3992 mindless
3993 obsession
3994 repellent
3995 sexual
3996 shame
3997 statistic
3998 together
3999 doc
4000 gum
4001 processor
4002 butt
4003 colon
4004 fan
4005 headline
4006 origonal
4007 rectify
4008 resent
4009 projector
4010 semd
4011 forwards
4012 njd
4013 oiler
4014 partial
4015 shooter
4016 soo
4017 capital
4018 semiconductor
4019 symptom
4020 xrdb
4021 xresource
4022 xterm
4023 chastity
4024 climate
4025 clinic
4026 incarcerate
4027 intellect
4028 precendent
4029 quarantine
4030 resistant
4031 sanitarium
4032 shameful
4033 skepticism
4034 sporadically
4035 surrender


5307 fidonet
5308 powerpc
5309 quadra
5310 chap
5311 agnostic
5312 alternate
5313 amusing
5314 bake
5315 geek
5316 polite
5317 shoulder
5318 whirrr
5319 wholesome
5320 banding
5321 disturb
5322 drawing
5323 gxxor
5324 inverse
5325 redraw
5326 underlying
5327 xsetfunction
5328 colour
5329 cook
5330 equipped
5331 gramming
5332 grayscale
5333 premier
5334 quickdraw
5335 signature
5336 tidewater
5337 blessing
5338 criterion
5339 remark
5340 anticipate
5341 grantor
5342 guilt
5343 inadvertantly
5344 innocence
5345 string
5346 uphold
5347 verdict
5348 removable
5349 cannibalize
5350 convertible
5351 outrageous
5352 immerse
5353 knowledgeably
5354 merit
5355 creation
5356 ecological
5357 uucp
5358 critical
5359 standout
5360 compelling
5361 considerably
5362 council
5363 enormous
5364 existance
5365 harmony
5366 remarkable
5367 tract
5368 binary
5369 taxis
5370 endometriosis
5371 adrift
5372 afterward
5373 altitude
5374 compensate
5375 costar
5376 grapple
5377 guidance
5378 gyro
5379 hst
5380

...

23303 bdftool
23304 bitmaps
23305 bldfamily
23306 combining
23307 cron
23308 cshrc
23309 decwrl
23310 displaymanager
23311 illegality
23312 interwork
23313 lc_ctype
23314 likeli
23315 lpr
23316 mkfontdir
23317 poskanzer
23318 prebuilt
23319 rsh
23320 screendump
23321 sdsc
23322 subshell
23323 umlaut
23324 unreferenced
23325 unrestricted
23326 unsetenv
23327 unspecified
23328 utoronto
23329 xgrabsc
23330 xloadimage
23331 xmh
23332 xpm
23333 xprinter
23334 xselection
23335 xset
23336 xsnap
23337 xtiff
23338 xwebster
23339 friendlyness
23340 gouroud
23341 airstrike
23342 ared
23343 beleagured
23344 coldhearted
23345 enthusiasm
23346 peacekeeping
23347 wre
23348 deshaie
23349 mahome
23350 pagliarulo
23351 trombley
23352 thiokol
23353 althoug
23354 runway
23355 chile
23356 tomatoe
23357 ketchup
23358 pickle
23359 islote
23360 exlcude
23361 origianl
23362 venial
23363 astral
23364 astrological
23365 coming
23366 eckankar
23367 etheric
23368 hypnotic
23369 hypnotist
23370 lowest
23371 transmu

24534 disgruntled
24535 wepaon
24536 descramble
24537 functionallity
24538 artur
24539 arturs
24540 evalute
24541 fide
24542 goonism
24543 irbe
24544 prompting
24545 undrafte
24546 calamari
24547 archiver
24548 bilevel
24549 blocky
24550 blurred
24551 blurriness
24552 buggier
24553 checkbox
24554 chrominance
24555 cjpeg
24556 colormappe
24557 colorview
24558 cray
24559 cropping
24560 cshow
24561 decodable
24562 decompressor
24563 denominator
24564 djpeg
24565 dvpeg
24566 filtering
24567 forevermore
24568 frill
24569 fullview
24570 gifconverter
24571 hiview
24572 hoopla
24573 indexing
24574 irretrievably
24575 jbig
24576 jpege
24577 jpegsrc
24578 jpegview
24579 kbytes
24580 microsystem
24581 milnet
24582 nonprofessional
24583 pictpixie
24584 pmjpeg
24585 pnmconvol
24586 precompute
24587 prepress
24588 qt
24589 quantize
24590 quantizer
24591 recompresse
24592 roundoff
24593 slideshow
24594 sloppiness
24595 spiffier
24596 subsample
24597 technologys
24598 transportable
24599 unusably
2460

25631 zumdahl
25632 chessboard
25633 chesspiece
25634 prejudgment
25635 repeated
25636 unsolvable
25637 salad
25638 movment
25639 orelativity
25640 eliass
25641 underpredict
25642 overnite
25643 rollerblader
25644 rucksack
25645 shifting
25646 surf
25647 carel
25648 spidery
25649 bundled
25650 macland
25651 outscore
25652 dewy
25653 mananger
25654 asinine
25655 reoccure
25656 gobs
25657 assad
25658 digita
25659 emirs
25660 gulf
25661 infidelity
25662 mistress
25663 mentionne
25664 packagaing
25665 nourish
25666 waif
25667 acetylene
25668 propane
25669 verboten
25670 supertuner
25671 impertinent
25672 imortant
25673 prosyletize
25674 electoral
25675 pavarottis
25676 shakeup
25677 usa
25678 dcw
25679 stubby
25680 olvwm
25681 commendation
25682 hamartia
25683 iota
25684 uncompromising
25685 aestetic
25686 bewilder
25687 sculptured
25688 tookit
25689 scandanavian
25690 sideline
25691 unmarried
25692 pesky
25693 unaccessible
25694 befuddlement
25695 entwine
25696 fertilize
25697 kilobyte
25

27053 dolphins
27054 indeterminism
27055 petri
27056 solipsism
27057 stony
27058 circularize
27059 cryogenically
27060 descoping
27061 sirtf
27062 classc
27063 uims
27064 visuallib
27065 acquaintace
27066 surviving
27067 roy
27068 salmon
27069 duplexer
27070 hardline
27071 stil
27072 synching
27073 sieger
27074 spotlight
27075 inherence
27076 contex
27077 preform
27078 twiddling
27079 voronoi
27080 densitometry
27081 micrograph
27082 egotistical
27083 globule
27084 shuold
27085 ramcheck
27086 ruffle
27087 reccurre
27088 deemphasize
27089 genealogical
27090 knowledgably
27091 messge
27092 complane
27093 stirring
27094 backround
27095 hershis
27096 lingo
27097 steamed
27098 appology
27099 empathic
27100 contardictory
27101 giveth
27102 taketh
27103 wince
27104 ifthe
27105 jurisdication
27106 kisser
27107 pike
27108 shedule
27109 monstrosity
27110 audibly
27111 judged
27112 yardstick
27113 nonserbian
27114 legalitie
27115 ssn
27116 interferance
27117 lurid
27118 overstatement
27119 robohe

28320 cx_scub
28321 gazebo
28322 gcx_
28323 gcx_g
28324 gcx_s
28325 gcx_sc
28326 glryxj
28327 iak_q
28328 mcx
28329 mcx_scx
28330 neccesary
28331 pieced
28332 qsw
28333 reguarding
28334 rfusv
28335 scx_s
28336 scx_scx
28337 simrmc
28338 sqs
28339 txr
28340 uucoder
28341 wxte
28342 x_g
28343 x_syx
28344 y
28345 yx_g
28346 yx_s
28347 yx_scx
28348 indefinable
28349 doper
28350 foriegn
28351 mecha
28352 asymmetry
28353 chiltepin
28354 commissioning
28355 moonlite
28356 niven
28357 sundae
28358 adlib
28359 amplified
28360 audition
28361 bagging
28362 czechoslovakian
28363 varialble
28364 intercessory
28365 lovingkindness
28366 aeronautic
28367 alluvium
28368 asteroids
28369 astrodynamic
28370 ellipsoidal
28371 minovitch
28372 parallax
28373 parallelize
28374 perubation
28375 topos
28376 trig
28377 trignometry
28378 polarising
28379 canonicity
28380 crosswire
28381 glassner
28382 reencrypted
28383 karate
28384 mallard
28385 raptor
28386 tailed
28387 cheapish
28388 arrese
28389 deference
2839

In [216]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)]]


In [222]:
[[(id2word[id],freq) for id, freq in cp] for cp in corpus[1:]]

[[('call', 2),
  ('day', 2),
  ('thank', 1),
  ('acceleration', 1),
  ('adapter', 1),
  ('add', 2),
  ('answer', 1),
  ('article', 1),
  ('attain', 1),
  ('base', 1),
  ('brave', 1),
  ('brief', 1),
  ('card', 1),
  ('clock', 4),
  ('detail', 1),
  ('disk', 1),
  ('do', 1),
  ('especially', 1),
  ('experience', 2),
  ('final', 2),
  ('floppy', 2),
  ('functionality', 1),
  ('guy', 1),
  ('heat', 1),
  ('hour', 1),
  ('keyword', 1),
  ('knowledge', 1),
  ('message', 1),
  ('network', 1),
  ('next', 1),
  ('number', 1),
  ('oscillator', 1),
  ('poll', 3),
  ('procedure', 1),
  ('rate', 1),
  ('report', 1),
  ('request', 1),
  ('send', 1),
  ('share', 1),
  ('si', 1),
  ('sink', 1),
  ('soul', 1),
  ('speed', 2),
  ('summarize', 1),
  ('summary', 1),
  ('top', 1),
  ('upgrade', 2),
  ('usage', 1)],
 [('could', 3),
  ('day', 1),
  ('info', 1),
  ('know', 1),
  ('line', 1),
  ('look', 2),
  ('make', 1),
  ('post', 1),
  ('really', 2),
  ('thank', 1),
  ('wonder', 1),
  ('answer', 1),
  ('di